In [1]:
from astroquery.gaia import Gaia
from astropy import units as u
from astropy.constants import h, c
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display

In [2]:
# Constants

d_filters = {'Band':['G','BP','RP'],'Wavelength':[639*u.nm,518*u.nm,783*u.nm],'Bandpass':[290*u.nm,169*u.nm,189*u.nm]} 
filters = pd.DataFrame(data=d_filters)

ccd_width_px = 4656
ccd_height_px = 3520
ccd_px_side_length_micron = 3.8 * u.micron
ccd_width_micron = ccd_width_px * ccd_px_side_length_micron
ccd_height_micron = ccd_height_px * ccd_px_side_length_micron
plate_scale_arcsec_micron = 0.11 * u.arcsec
ccd_width_range_arcsec = ccd_width_micron * plate_scale_arcsec_micron
ccd_height_range_arcsec = ccd_height_micron * plate_scale_arcsec_micron
ccd_width_range_deg = u.arcsec.to(u.deg, value=ccd_width_range_arcsec)
ccd_height_range_deg = u.arcsec.to(u.deg, value=ccd_height_range_arcsec)
mast_collecting_area_m2 = 0.09 * np.pi * 0.5 * u.m**2
gaia_collecting_area_m2 = 0.7 * u.m**2

In [3]:
ra_widget = widgets.BoundedFloatText(
    value=279.235,
    min=0,
    max=360,
    step=0.001,
    description='RA:'
)
dec_widget = widgets.BoundedFloatText(
    value=38.783,
    min=-90,
    max=90,
    step=0.001,
    description='DEC:'
)
max_mag_widget = widgets.FloatText(
    value=18,
    step=0.01,
    description='Max Mag:'
)
display(ra_widget)
display(dec_widget)
display(max_mag_widget)

BoundedFloatText(value=279.235, description='RA:', max=360.0, step=0.001)

BoundedFloatText(value=38.783, description='DEC:', max=90.0, min=-90.0, step=0.001)

FloatText(value=18.0, description='Max Mag:', step=0.01)

In [4]:
def get_entries(max_mag, ra, dec, ccd_width_range_deg, ccd_height_range_deg):
    query = '''SELECT TOP 100 RA, DEC,
PHOT_G_MEAN_FLUX AS PHOT_G_MEAN_FLUX_GAIA, PHOT_G_MEAN_FLUX_ERROR AS PHOT_G_MEAN_FLUX_ERROR_GAIA,
PHOT_BP_MEAN_FLUX AS PHOT_BP_MEAN_FLUX_GAIA, PHOT_BP_MEAN_FLUX_ERROR AS PHOT_BP_MEAN_FLUX_ERROR_GAIA,
PHOT_RP_MEAN_FLUX AS PHOT_RP_MEAN_FLUX_GAIA, PHOT_RP_MEAN_FLUX_ERROR AS PHOT_RP_MEAN_FLUX_ERROR_GAIA
FROM gaiadr3.gaia_source
WHERE (ra BETWEEN {ra_min} AND {ra_max})
AND (dec BETWEEN {dec_min} AND {dec_max})
AND phot_g_mean_mag < {max_mag}
'''.format(
        max_mag=max_mag,
        ra_min=ra-(ccd_width_range_deg.value/2), ra_max=ra+(ccd_width_range_deg.value/2),
        dec_min=dec-(ccd_height_range_deg.value/2), dec_max=dec+(ccd_height_range_deg.value/2)
    )
    job = Gaia.launch_job(query)
    r = job.get_results()
    return r

In [43]:
def convert_to_flux_density(table):
    # gets flux mean, flux error in electrons/sec (as given in GAIA_SOURCE) and name of band {'G','BP,'RP'}
    # returns flux density + uncertainty in watt/m^2/nm
    for filter in d_filters['Band']:
        lam0 = filters[filters.Band == filter].iloc[0]['Wavelength']
        bandpass = filters[filters.Band == filter].iloc[0]['Bandpass']
        P =  h * c /lam0.to(u.m)/u.s*table['phot_' + str.lower(filter) + '_mean_flux_gaia'].value # convert rate of elecron arrival, to incoming power in watts 
        dP = np.sqrt((h * c /lam0.to(u.m)*table['phot_' + str.lower(filter) + '_mean_flux_error_gaia'].value)**2  + (bandpass.to(u.m)/2 * table['phot_' + str.lower(filter) + '_mean_flux_gaia'].value * h * c /lam0.to(u.m)**2)**2)/u.s
        F = P/gaia_collecting_area_m2 # convert incoming power, to incoming intensity- 0.7m^2 is GAIA's collecting area
        dF = dP/gaia_collecting_area_m2
        table['flux_' + filter + '_density_gaia'] = (F/bandpass.to(u.nm)) # convert to incoming flux density
        table['flux_' + filter + '_density_error_gaia'] = dF/bandpass.to(u.nm) 

In [47]:
def calculate_mast_qe(flux_density_inc_mast):
    return flux_density_inc_mast

In [45]:
def convert_gaia_to_mast(table):
    for filter in d_filters['Band']:
        lam0 = filters[filters.Band == filter].iloc[0]['Wavelength']
        bandpass = filters[filters.Band == filter].iloc[0]['Bandpass']
        gaia_mast_attenuation = 0.5
        flux_density_inc_mast =  gaia_mast_attenuation * table['flux_' + filter + '_density_gaia']
        flux_meas_mast = calculate_mast_qe(flux_density_inc_mast) * bandpass
        power_meas_mast = flux_meas_mast * mast_collecting_area_m2
        table['phot_' + filter + '_flux_mast'] = (power_meas_mast * lam0.to(u.m) / (h * c)) * u.electron 

        flux_density_inc_error_mast =  gaia_mast_attenuation * table['flux_' + filter + '_density_error_gaia']
        flux_meas_error_mast = calculate_mast_qe(flux_density_inc_error_mast) * bandpass
        power_meas_error_mast = flux_meas_error_mast * mast_collecting_area_m2
        table['phot_' + filter + '_flux_rel_error_mast'] = ((power_meas_error_mast * lam0.to(u.m) / (h * c)).value / table['phot_' + filter + '_flux_mast']).value * 100 * u.percent

In [49]:
filters

,Band,Wavelength,Bandpass
0,G,639.0 nm,290.0 nm
1,BP,518.0 nm,169.0 nm
2,RP,783.0 nm,189.0 nm


In [48]:
entries = get_entries(max_mag_widget.value, ra_widget.value, dec_widget.value, ccd_width_range_deg, ccd_height_range_deg)
convert_to_flux_density(entries)
convert_gaia_to_mast(entries)
entries

ra,dec,phot_g_mean_flux_gaia,phot_g_mean_flux_error_gaia,phot_bp_mean_flux_gaia,phot_bp_mean_flux_error_gaia,phot_rp_mean_flux_gaia,phot_rp_mean_flux_error_gaia,flux_G_density_gaia,flux_G_density_error_gaia,flux_BP_density_gaia,flux_BP_density_error_gaia,flux_RP_density_gaia,flux_RP_density_error_gaia,phot_G_flux_mast,phot_G_flux_rel_error_mast,phot_BP_flux_mast,phot_BP_flux_rel_error_mast,phot_RP_flux_mast,phot_RP_flux_rel_error_mast
deg,deg,electron / s,electron / s,electron / s,electron / s,electron / s,electron / s,J / (m2 nm s),J / (m2 nm s),J / (m2 nm s),J / (m2 nm s),J / (m2 nm s),J / (m2 nm s),electron / s,%,electron / s,%,electron / s,%
float64,float64,float64,float32,float64,float32,float64,float32,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
278.96494437636767,38.658773196656206,4302.854510364426,2.3290374,2296.7434898592805,13.387459,2837.5006055939084,8.756109,6.589257781191359e-18,1.4952192432848029e-18,7.445171771206538e-18,1.2152866983902221e-18,5.441154891635843e-18,6.56905727771497e-19,434.50123240371175,22.691770347076364,231.92461526535317,16.323151912897735,286.53014112076323,12.072909903397422
278.96743756702864,38.63946435621336,2657.865941639628,1.8847753,1428.7742809720794,13.07814,1909.039869012362,18.53691,4.070173368661991e-18,9.235962758800728e-19,4.6315446157076056e-18,7.567203727649466e-19,3.6607504509872385e-18,4.432423328622077e-19,268.39067517262197,22.69181659413421,144.27728950949756,16.338401884299568,192.77439518247613,12.107963621029489
278.9684014504069,38.778231947989305,1646.1522057227946,1.5369931,781.7075520014079,13.571034,1260.1791223368552,8.712325,2.5208663700938324e-18,5.720324224017826e-19,2.5339995629448514e-18,4.156991241542655e-19,2.4165033770646057e-18,2.9212507729909473e-19,166.22806102037248,22.691897880350172,78.93664401289128,16.404861714781305,127.25259020166527,12.088751047140983
278.96936300233864,38.79932451902017,6810.431011055317,3.0891402,3268.1938880109233,10.967302,4999.242220504968,11.198962,1.042928256690289e-17,2.366586844179611e-18,1.0594245715850264e-17,1.7285775354862154e-18,9.58648298046074e-18,1.1571886076548912e-18,687.7157153178694,22.691751124760245,330.0214470776414,16.316192599724737,504.82229893246074,12.07104430283227
278.9697701690212,38.76867487527962,5362.227006938642,3.1430779,2732.7030882092304,12.74669,3955.5813417666595,11.816019,8.211547925888947e-18,1.8633465126875507e-18,8.858387530542579e-18,1.4456364770045337e-18,7.585172219729706e-18,9.157321825587471e-19,541.4764169532608,22.691781494849312,275.94771256157975,16.31940883168822,399.4336698419248,12.072661714612707
278.9701077866078,38.81495235657511,1549.952337287637,1.7168857,752.1964547043827,7.3966246,1131.6151908605216,10.858532,2.3735488788542176e-18,5.38605145562778e-19,2.4383357722328103e-18,3.984814233387905e-19,2.1699708254023055e-18,2.627194754641725e-19,156.51382102190902,22.6919761527211,75.95662037642926,16.342352348540448,114.27023475957942,12.107051043668529
278.97203665414145,38.94954970762994,4278.840297814535,2.5266147,2302.145087092561,12.499132,2822.9228735319343,10.011038,6.552483161802671e-18,1.4868752352745938e-18,7.462681701861925e-18,1.2180420419502907e-18,5.413200818970063e-18,6.535993216711898e-19,432.0762828910106,22.691782619789834,232.47006727846576,16.32177400312265,285.05808517945223,12.074174661703134
278.97219384397613,38.634540142284195,37193.95593424609,7.079606,19418.43668519263,20.070137,25384.179991621157,18.89729,5.695766913863485e-17,1.292467125298613e-17,6.294721081735407e-17,1.026862176865277e-17,4.867637908491824e-17,5.874847166916119e-18,3755.8368874597513,22.691713773482387,1960.869151105705,16.31306873698967,2563.2885014703656,12.069195115493638
